In [2]:
%matplotlib inline

import pandas as pd
import numpy as np

import glob
import re

import matplotlib.pyplot as plt

from collections import Counter

### Read in the Data and a bit of Clean Up

In [3]:
###########################################
# create a function to combine data files #
###########################################

def frame_masher():
    # find all of the query files
    numbers = [1,4,5,6,7,11,12,15] # combine these datafiles
    query_files = []
    for num in numbers:
        file ='data/tea_party/tea_party{}.csv'.format(num)
        query_files.append(file)
    
    
    # empty list to add data frames to
    frame_list = []
    
    # iterate through the files to create one combined date frame
    for frame in query_files:
        df = pd.read_csv(frame)
        frame_list.append(df)
    joined = pd.concat(frame_list).reset_index()
    final = joined.drop('index',axis = 1)
    return final

In [4]:
# create the combined data frame
df = frame_masher()

# little bit of clean up
df = df.loc[df['job_id'] == '2572']

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,3,4,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,1,2,3,4,5,6,9,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (3,4,5,6,9,11,12,14) have mixed types. Speci

In [6]:
df['created_at']=pd.to_datetime(df['created_at'],format='%Y-%m-%d %H:%M:%S')
df['Month'] = df['created_at'].dt.month
df['Year'] = df['created_at'].dt.year
df['Day'] = df['created_at'].dt.day

### Clean up tweets

In [16]:
# Pre-process tweets

def processTweet(tweet):
    # process the tweets
    tweet = str(tweet)
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end

In [19]:
# create a function to iterate over an entire column of tweets and apply the processTweet function
def tweetCleaner(column):
    tweets = []
    for text in column:
        cleaned = processTweet(text)
        tweets.append(cleaned)
    return tweets

In [21]:
# write result to the data frame
df['tweet']= tweetCleaner(df['text'])

### Aggregate by how many terms per month

In [68]:
# here is a function that will count the occurances of a column per month and return a data frame

def dataAggregator(df,column,name):
    new = df.groupby(['Year','Month','Day'])[column].agg({'no_{}'.format(name):'count'})
    new.reset_index(level=[0,1,2], inplace=True)
    new['datetime'] = pd.to_datetime((new.Year*10000+new.Month*100+new.Day).apply(str),format='%Y%m%d')
    return new

In [48]:
# makes some dataframes that contain key words

obama =df[df["tweet"].str.contains("obama")]
stupid =df[df["tweet"].str.contains("stupid")]
liberal =df[df["tweet"].str.contains("liberal")] 
conservative =df[df["tweet"].str.contains("conservative")]
idiot = df[df["tweet"].str.contains("idiot")] 
values = df[df["tweet"].str.contains("values")] 
principles = df[df["tweet"].str.contains("principles")]

In [69]:
# count the number of tweets per month for each of the data frames above

tot = dataAggregator(df,'job_id','tot') # all tweets per month
stupid_month = dataAggregator(stupid,'job_id','stupid')
obama_month = dataAggregator(obama,'job_id','obama')
liberal_month = dataAggregator(liberal,'job_id','liberal')
idiot_month = dataAggregator(idiot,'job_id','idiot')
conservative_month = dataAggregator(conservative,'job_id','conservative')
values_month = dataAggregator(values,'job_id','values')
principles_month = dataAggregator(principles,'job_id','principles')

In [74]:
# merge all of these dataframes in a very ugly way

first = tot.merge(stupid_month, on = ['datetime','Day','Month','Year'])
second = first.merge(obama_month, on= ['datetime','Day','Month','Year'] )
third = second.merge(liberal_month, on= ['datetime','Day','Month','Year'] )
fourth = third.merge(idiot_month, on= ['datetime','Day','Month','Year'] )
fifth = fourth.merge(conservative_month, on= ['datetime','Day','Month','Year'] )
sixth = fifth.merge(values_month, on= ['datetime','Day','Month','Year'] )
seventh = sixth.merge(principles_month, on= ['datetime','Day','Month','Year'] )

In [79]:
# convert the features into ratios 

seventh['stupid_rate'] = seventh['no_stupid']/seventh['no_tot']
seventh['obama_rate']= seventh['no_obama']/seventh['no_tot']
seventh['liberal_rate']= seventh['no_liberal']/seventh['no_tot']
seventh['idiot_rate']= seventh['no_idiot']/seventh['no_tot']
seventh['conservative_rate']= seventh['no_conservative']/seventh['no_tot']
seventh['values_rate']= seventh['no_values']/seventh['no_tot']
seventh['principles_rate']= seventh['no_principles']/seventh['no_tot']

In [80]:
seventh

,Year,Month,Day,no_tot,datetime,no_stupid,no_obama,no_liberal,no_idiot,no_conservative,no_values,no_principles,stupid_rate,obama_rate,liberal_rate,idiot_rate,conservative_rate,values_rate,principles_rate
0,2014,8,5,5819,2014-08-05,17,383,53,23,268,2,2,0.002921,0.065819,0.009108,0.003953,0.046056,0.000344,0.000344
1,2014,8,6,13722,2014-08-06,30,638,77,29,290,9,12,0.002186,0.046495,0.005611,0.002113,0.021134,0.000656,0.000875
2,2014,8,7,10057,2014-08-07,24,770,69,21,188,1,3,0.002386,0.076564,0.006861,0.002088,0.018693,0.000099,0.000298
3,2014,8,8,9876,2014-08-08,45,414,81,38,211,9,4,0.004557,0.041920,0.008202,0.003848,0.021365,0.000911,0.000405
4,2014,8,9,7163,2014-08-09,21,421,66,34,139,2,2,0.002932,0.058774,0.009214,0.004747,0.019405,0.000279,0.000279
5,2014,8,10,6632,2014-08-10,13,408,50,15,178,36,1,0.001960,0.061520,0.007539,0.002262,0.026840,0.005428,0.000151
6,2014,8,11,8693,2014-08-11,26,418,49,19,240,6,1,0.002991,0.048085,0.005637,0.002186,0.027608,0.000690,0.000115
7,2014,8,12,8567,2014-08-12,22,355,59,15,159,10,3,0.002568,0.041438,0.006887,0.001751,0.018560,0.001167,0.000350
8,2014,8,13,9300,2014-08-13,19,273,160,12,147,1,5,0.002043,0.029355,0.017204,0.001290,0.015806,0.000108,0.000538
9,2014,8,14,15388,2014-08-14,35,673,108,33,445,6,15,0.002274,0.043735,0.007018,0.002145,0.028919,0.000390,0.000975
